In [ ]:
import ollama
import requests
import json

In [2]:
model = "llama3.2:1b"

#### Task 1: Interact with deployed LLM via python 


**Objective:**

Explore different techniques to interact with the deployed LLM.

**Task Description:**

1. Use Request libaray (HTTP Client) and send a POST request to interact with the LLM: [How To](https://requests.readthedocs.io/en/latest/user/quickstart/#make-a-request)

In [3]:
# Simple HTTP Request via requests

# Define the URL of the deployed LLM ( this port is forwarded from the docker container to the host system)
url = "http://localhost:11434/api/generate"

# Define the prompt as json
body_json = {
    "model": model,
    "prompt": "Describe Generative AI in two sentences."
}

# ADD HERE YOUR CODE
# HINT: Send the POST request using the json body
response = requests.post(url, json=body_json)

# Check if the request was successful
if response.status_code == 200:
    # Process the response
    response_text = response.text

    # Convert each line to json
    response_lines = response_text.splitlines()
    response_json = [json.loads(line) for line in response_lines]
    for line in response_json:
        # Print the response. No line break
        print(line["response"], end="")
else:
    print("Error:", response.status_code)


Generative AI refers to a class of machine learning algorithms that can create new data samples or text based on patterns and structures learned from existing data, often producing highly realistic and diverse outputs. Unlike traditional machine learning models, generative AI uses complex neural networks to generate novel content, such as images, videos, music, and even entire stories, without being explicitly programmed for a specific task.

**Task Description:**

2. Use Ollama python library to interact with the LLM: [How To](https://pypi.org/project/ollama/)

- First use method ``ollama.chat(...)``
- First use method ``ollama.chat(...)`` with ``stream=True``

In [4]:
# API Call via ollama

# ADD HERE YOUR CODE

messages = [
    {"role": "user", "content": "Describe Generative AI in two sentences."}
]

response = ollama.chat(
    model=model,
    messages=messages
)


print(response["message"]["content"])

Generative AI is a subfield of artificial intelligence that enables machines to create original content, such as text, images, and music, using patterns and algorithms learned from existing data or training examples. By generating new content that resembles the style of human-provided examples, generative AI can perform tasks like image editing, chatbot generation, and even creative writing, but it often lacks the nuance and emotional depth found in human-generated content.


In [5]:
# Streaming API Call via ollama

# Response streaming can be enabled by setting stream=True, 
# modifying function calls to return a Python generator where each part is an object in the stream.

# ADD HERE YOUR CODE

stream = ollama.chat(
    model=model,
    messages=messages,
    stream=True
)


for chunk in stream:
  print(chunk["message"]["content"], end="", flush=True)

Generative AI refers to a type of artificial intelligence that can create original content, such as text, images, or music, by generating new data based on patterns and associations learned from existing data. Unlike traditional machine learning models that focus on predicting outcomes, generative AI systems aim to produce novel and realistic outputs that resemble the real world, often with high levels of creativity and nuance.

#### Task 2: Experimenting with Prompt Techniques

**Objective:**

Objective: Explore different prompt techniques (Zero Shot, One Shot, and Few Shot) by sending different types of prompts to the LLM.

![image](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*QSpK--jqPiUU_OHuZvtUWA.png)

**Task Description:**

1. Create three prompts for a sentiment analysis task: a Zero Shot prompt, a One Shot prompt, and a Few Shot prompt. Use the examples from the table above.
2. Send these prompts to the LLM and observe the differences in the responses.
3. Compare and discuss the responses.

In [7]:
# ADD HERE YOUR PROMPTS

zero_shot_prompt = """Classify this review: I loved this movie! Sentiment:"""

one_shot_prompt =(
    'Classify this review: I loved this movie! Sentiment: Positive\n'
    "Classify this review: I don't like this chair. Sentiment:"
)

few_shot_prompt = (
    'Classify this review: I loved this movie! Sentiment: Positive\n'
    "Classify this review: I don't like this chair. Sentiment: Negative\n"
    'Classify this review: Who would use this product? Sentiment:'
)

# Stream the responses and print them
for idx, prompt in enumerate([zero_shot_prompt, one_shot_prompt, few_shot_prompt]):
    prompt_type = ["Zero-Shot", "One-Shot", "Few-Shot"][idx]
    print(f"\n--- {prompt_type} Prompt ---\n")
    print(f"User Prompt:\n{prompt}\n")
    
    stream = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )
    
    print("Model Output:")
    for chunk in stream:
        print(chunk["message"]["content"], end="", flush=True)
    print("\n-----------------------------\n")



--- Zero-Shot Prompt ---

User Prompt:
Classify this review: I loved this movie! Sentiment:

Model Output:
The sentiment of this review is extremely positive.
-----------------------------


--- One-Shot Prompt ---

User Prompt:
Classify this review: I loved this movie! Sentiment: Positive
Classify this review: I don't like this chair. Sentiment:

Model Output:
For the first review, "I loved this movie," the sentiment is overwhelmingly positive.

For the second review, "I don't like this chair," the sentiment is negative.
-----------------------------


--- Few-Shot Prompt ---

User Prompt:
Classify this review: I loved this movie! Sentiment: Positive
Classify this review: I don't like this chair. Sentiment: Negative
Classify this review: Who would use this product? Sentiment:

Model Output:
Based on the content, here are my classifications:

1. "I loved this movie!"
   - Sentiment: Positive
2. "I don't like this chair."
   - Sentiment: Negative (Note: This one is a very brief and str

#### Task 3: Prompt Refinement and Optimization

**Objective:** 

Refine a prompt to improve the clarity and quality of the LLM's response.

**Task Description:**

- Start with a basic prompt asking the LLM to summarize a paragraph.
- Refine the prompt by adding specific instructions to improve the summary's quality. (Example: define how long the summary should be, define on which to focus in the summary)

In [8]:
# Original prompt
original_prompt = "Summarize the following paragraph: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries."

# ADD HERE YOUR PROMPT
refined_prompt = (
    "Summarize the following paragraph in 2 concise sentences, focusing on the main goal "
    "of Generative AI and its key application areas. Avoid repeating the original phrasing "
    "and keep the tone informative and professional.\n\n"
    "Paragraph: Generative AI is a field of artificial intelligence focused on creating new "
    "content based on patterns learned from existing data. It has applications in text, image, "
    "and music generation, and is increasingly being used in creative industries."
)

# Stream the responses and print them
for idx, prompt in enumerate([original_prompt, refined_prompt]):
    prompt_type = ["Original Prompt", "Refined Prompt"][idx]
    print(f"\n--- {prompt_type} ---\n")
    print(f"User Prompt:\n{prompt}\n")
    
    stream = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )
    
    print("Model Output:")
    for chunk in stream:
        print(chunk["message"]["content"], end="", flush=True)
    print("\n-----------------------------\n")



--- Original Prompt ---

User Prompt:
Summarize the following paragraph: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries.

Model Output:
Here's a summary of the paragraph:

Generative AI creates new content by analyzing patterns from existing data, such as text, images, or music, and uses that knowledge to produce original works. It has applications across various fields, including creative industries.
-----------------------------


--- Refined Prompt ---

User Prompt:
Summarize the following paragraph in 2 concise sentences, focusing on the main goal of Generative AI and its key application areas. Avoid repeating the original phrasing and keep the tone informative and professional.

Paragraph: Generative AI is a field of artificial intelligence focused on creating new content based on pattern

#### [Optional] Task 4: Structured Prompting with Roles (Pirate Theme)

**Objective:**

Learn how to use structured prompts that combine role assignment, clear instructions, and examples to improve the output of language models. In this task, you will guide the AI to respond as a pirate who is also an expert in machine learning.

**Instructions:**

- Role Assignment: In your prompt, specify the role of the AI as a Machine Learning Expert who speaks like a pirate.

- Instruction: Clearly state what you want the AI to explain or discuss in pirate language.

- Examples: Provide examples to guide the AI in using pirate lingo while explaining technical concepts.

In [ ]:
# Combined Techniques Prompt with Pirate Theme

structured_prompt = """
"""

# Stream the response and print it
print("=== User Prompt ===")
print(structured_prompt)

stream = ollama.chat(
    model=model,
    messages=[{"role": "user", "content": structured_prompt}],
    stream=True,
)

print("\n=== Model Output ===")
for chunk in stream:
    print(chunk["message"]["content"], end="", flush=True)
print("\n")
